In [193]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [7]:
table = """
Gate	gRNA (rset)	Circuit	Part	Status	
BE	r5	GANDER NOR (GOOD)	pMOD8-pGALZ4-URGR-W17	Minipreps	
DOX	r9	GANDER NOR (GOOD)	TRP<7XTetO-pMinCyc1-URGR-W34-tCyc1>	Submitted	
Reporter	r5-r9-GFP	GANDER NOR (GOOD)	pGRR-W17W34-yeGFP		
BE	r10	GANDER NOR (BAD)	pMOD8-pGALZ4-URGR-W36	Minipreps	
DOX	r1	GANDER NOR (BAD)	TRP<7XTetO-pMinCyc1-URGR-W5-tCyc1>	Submitted	
Reporter	r10-r1-GFP	GANDER NOR (BAD)	pGRR-W36W5-yeGFP		
BE	r2	GANDER OR (GOOD)	pMOD8-pGALZ4-URGR-W8	Minipreps	
DOX	r9	GANDER OR (GOOD)	TRP<7XTetO-pMinCyc1-URGR-W34-tCyc1>	Submitted	
Gate	r2-r9-r5	GANDER OR (GOOD)	pGRR-W8W34-RGR-W17		
Reporter	r5-r5-GFP	GANDER OR (GOOD)	pGRR-W17W17-yeGFP		
BE	r1	GANDER OR (BAD)	pMOD8-pGALZ4-URGR-W5	Minipreps	
DOX	r2	GANDER OR (BAD)	TRP<7XTetO-pMinCyc1-URGR-W8-tCyc1>	Built	
Gate	r1-r2-r10	GANDER OR (BAD)	pGRR-W5W8-RGR-W36		
Reporter	r10-r10-GFP	GANDER OR (BAD)	pGRR-W36W36-yeGFP		
BE	r2	DSGRN NOR (GOOD)	pMOD8-pGALZ4-URGR-W8	Minipreps	
DOX	r9	DSGRN NOR (GOOD)	TRP<7XTetO-pMinCyc1-URGR-W34-tCyc1>	Submitted	
Gate	r2-r9-r5	DSGRN NOR (GOOD)	pGRR-W8W34-RGR-W17		
Gate	r5-r5-r3	DSGRN NOR (GOOD)	pGRR-W17W17-RGR-W10		
Gate	r5-r5-r7	DSGRN NOR (GOOD)	pGRR-W17W17-RGR-W20		
Reporter	r3-r7-GFP	DSGRN NOR (GOOD)	pGRR-W10W20-yeGFP		
BE	r3	DSGRN NOR (BAD)	pMOD8-pGALZ4-URGR-W10	Minipreps	
DOX	r2	DSGRN NOR (BAD)	TRP<7XTetO-pMinCyc1-URGR-W8-tCyc1>	Built	
Gate	r3-r2-r1	DSGRN NOR (BAD)	pGRR-W10W8-RGR-W5		
Gate	r1-r1-r7	DSGRN NOR (BAD)	pGRR-W5W5-RGR-W20		
Gate	r1-r1-r10	DSGRN NOR (BAD)	pGRR-W5W5-RGR-W36		
Reporter	r7-r10-GFP	DSGRN NOR (BAD)	pGRR-W20W36-yeGFP		
BE	r2	DSGRN OR (GOOD)	pMOD8-pGALZ4-URGR-W8	Minipreps	
DOX	r9	DSGRN OR (GOOD)	TRP<7XTetO-pMinCyc1-URGR-W34-tCyc1>	Submitted	
Gate	r2-r9-r3	DSGRN OR (GOOD)	pGRR-W8W34-RGR-W10		
Gate	r2-r9-r5	DSGRN OR (GOOD)	pGRR-W8W34-RGR-W17		
Reporter	r3-r5-GFP	DSGRN OR (GOOD)	pGRR-W10W17-yeGFP		
BE	r3	DSGRN OR (BAD)	pMOD8-pGALZ4-URGR-W10	Minipreps	
DOX	r2	DSGRN OR (BAD)	TRP<7XTetO-pMinCyc1-URGR-W8-tCyc1>	Built	
Gate	r3-r2-r1	DSGRN OR (BAD)	pGRR-W10W8-RGR-W5		
Gate	r3-r2-r10	DSGRN OR (BAD)	pGRR-W10W8-RGR-W36		
Reporter	r1-r10-GFP	DSGRN OR (BAD)	pGRR-W5W36-yeGFP		
"""

with open('table.tmp.csv', 'w') as f:
    f.write(table)
    
import pandas as pd

with open('table.tmp.csv', 'r') as f:
    df = pd.read_csv(f, delimiter='\t')
df.head()

,Gate,gRNA (rset),Circuit,Part,Status,Unnamed: 5
0,BE,r5,GANDER NOR (GOOD),pMOD8-pGALZ4-URGR-W17,Minipreps,NaN
1,DOX,r9,GANDER NOR (GOOD),TRP<7XTetO-pMinCyc1-URGR-W34-tCyc1>,Submitted,NaN
2,Reporter,r5-r9-GFP,GANDER NOR (GOOD),pGRR-W17W34-yeGFP,NaN,NaN
3,BE,r10,GANDER NOR (BAD),pMOD8-pGALZ4-URGR-W36,Minipreps,NaN
4,DOX,r1,GANDER NOR (BAD),TRP<7XTetO-pMinCyc1-URGR-W5-tCyc1>,Submitted,NaN


In [17]:
wset_dict = """
W5	r1
W8	r2
W10	r3
W14	r4
W17	r5
W19	r6
W20	r7
W22	r8
W34	r9
W36	r10
F1	r11
F2	r12
F3	r13
F4	r14
F5	r15
F6	r16
F7	r17
F8	r18
F9	r19
F10	r20
""".strip().splitlines()
wset_to_rset_dict = {a: b for a, b in [x.split('\t') for x in wset_dict]}
rset_to_wset_dict = {v: k for k, v in wset_to_rset_dict.items()}

In [258]:
backbones = {
    'TRP': {
        'locus': 'TRP',
        'marker': 'TRP',
        'marker_type': 'auxotrophic',
        'fragments': (
            "TRP-pp2_5'hom__v2",
            "tCyc1(partial)-TS-TRP_3'hom__v3",
        )
    },
    'LEU': {
        'locus': 'LEU',
        'marker': 'LEU',
        'marker_type': 'auxotrophic',
        'fragments': (
            "LEU-pp2_5'hom__v3",
            "3'-hom-LEU"
        )
    },
    'URA': {
        'locus': 'URA',
        'marker': 'URA',
        'marker_type': 'auxotrophic',
        'fragments': (
            "URA-pp2_5'hom",
            "tCyc1(partial)-TS-URA_3'hom__v3",
        )
    },
    'HIS': {
        'locus': 'HIS',
        'marker': 'HIS',
        'marker_type': 'auxotrophic',
        'fragments': (
            "HIS-pp2_5'hom",
            "tCyc1(partial)-TS-HIS_3'hom__v3",
        )
    },
    'ARS': {
        'locus': 'ARS314',
        'marker': 'bleo',
        'marker_type': 'antibiotic',
        'fragments': (
            "ARS314-pp2_5'hom__v3",
            "TS-ARS314-BLEO_3'hom__v3"
        )
    },
    'HO': {
        'locus': 'HO',
        'marker': 'KAN',
        'marker_type': 'antibiotic',
        'fragments': (
            "HO-KAN-pp2_5'hom__v3",
            "TS-HO_3'hom__v3"
        )
    }
}

for b in backbones.values():
    frags = b['fragments']
    assert len(frags) == 2
    for f in frags:
        print(f, end=': ')
        assert aq.Sample.find_by_name(f)
        print('found')

TRP-pp2_5'hom__v2: found
tCyc1(partial)-TS-TRP_3'hom__v3: found
LEU-pp2_5'hom__v3: found
3'-hom-LEU: found
URA-pp2_5'hom: found
tCyc1(partial)-TS-URA_3'hom__v3: found
HIS-pp2_5'hom: found
tCyc1(partial)-TS-HIS_3'hom__v3: found
ARS314-pp2_5'hom__v3: found
TS-ARS314-BLEO_3'hom__v3: found
HO-KAN-pp2_5'hom__v3: found
TS-HO_3'hom__v3: found


In [202]:
from pprint import pprint

def part_component(x):
    c = None
    if x.startswith('r'):
        c = {
            'type': 'gRNA',
            'name': rset_to_wset_dict[x],
            'alias': x
        }
    else:
        c = {
            'type': 'protein',
            'name': x,
            'alias': None
        }
    return c

circuits = {}

dox_tf = {
    'gate': 'dox_tf',
    'backbone': backbones['HO'],
    'haploid': 'Mat A'
}

be_tf = {
    'gate': 'be_tf',
    'backbone': backbones['HO'],
    'haploid': 'Mat Alpha'
}

dcas9 = {
    'gate': 'dCas9-Mxi1',
    'backbone': backbones['TRP'],
    'haploid': 'Mat A'
}

for g, d in df.groupby('Circuit'):
    
    parts = [
        dox_tf,
        be_tf,
        dcas9
    ]
    for _, row in d.iterrows():
        gRNAs = row['gRNA (rset)'].split('-')
        inputs = gRNAs[:-1]
        outputs = gRNAs[-1:]
        
        inputs = [part_component(x) for x in inputs]
        outputs = [part_component(x) for x in outputs]
        part = {
            'gate': row.Gate,
            'inputs': inputs,
            'outputs': outputs,
            'backbone': None,
            'haploid': None
        }
        
        if part['gate'] == 'BE':
            part['backbone'] = backbones['HIS']
            part['haploid'] = 'Mat Alpha'
        elif part['gate'] == 'DOX':
            part['backbone'] = backbones['HIS']
            part['haploid'] = 'Mat A'
        elif part['gate'] == 'Reporter':
            part['backbone'] = backbones['URA']
            part['haploid'] = 'Mat Alpha'
        
        
        parts.append(part)
        
    circuits[g] = parts
#     for _, row in d.iterrows():
        
#     circuits[g] = d['']

# pprint(circuits)

In [257]:
from copy import deepcopy

def get_remaining_markers(markers, backbones):
    possible_markers = {
        'Mat A': [b['marker'] for b in backbones.values()]
    }
    possible_markers['Mat Alpha'] = possible_markers['Mat A'][:]

    remaining_markers = {}
    for mtype, _markers in markers.items():
        _markers = set(_markers[:])
        _all_markers = set(possible_markers[mtype][:])

        _remaining_markers = _all_markers.difference(_markers)
        remaining_markers[mtype] =_remaining_markers

    return remaining_markers

def iterate_markers(markers_dict, num_parts):
    for a, b in list(itertools.permutations(list(range(1, num_parts)), r=2)):
        l1 = itertools.combinations(markers_dict['Mat A'], r=a)
        l2 = itertools.combinations(markers_dict['Mat Alpha'], r=b)

        for m1, m2 in itertools.product(l1, l2):
            yield {'Mat A': m1, 'Mat Alpha': m2}

def valid_diploid_markers(l1, l2):
    s1 = set(l1)
    s2 = set(l2)
    if len(s1) != len(l1):
        return False
    if len(s2) != len(l2):
        return False
    i1 = s1.difference(s2)
    i2 = s2.difference(s1)
    if i1 and i2:
        return True
    return False


def iterate_valid_marker_selection(markers, num_parts, backbones):
    rmarkers = get_remaining_markers(markers, backbones)

    for d in iterate_markers(rmarkers, 3):
        all_markers = deepcopy(markers)
        for k, v in d.items():
            all_markers.setdefault(k, list())
            all_markers[k].extend(v)
            all_markers[k]

        a = all_markers['Mat A']
        b = all_markers['Mat Alpha']
        if valid_diploid_markers(a, b):
            yield {
                'selection': d, 'all': all_markers
            }
            
def iter_key_items(d):
    for k, v in d.items():
        for _v in v:
            yield k, _v
            
def find_backbone(marker, backbones):
    for k, b in backbones.items():
        if b['marker'] == marker:
            return k, b
        
def get_needs_assigment(parts):
    needs_assignment = []
    for part in parts:
        if part['backbone'] is None:
            needs_assignment.append(part)
    return needs_assignment

def assign_markers(example, n=1, mode='first'):
    example = deepcopy(example)
    markers = {}
    needs_assignment = get_needs_assigment(example)
    for part in example:
        if part['backbone'] is not None:
            mtype = part['haploid']
            assert mtype
            markers.setdefault(mtype, list())
            markers[mtype].append(part['backbone']['marker'])
    markers = list(iterate_valid_marker_selection(markers, len(needs_assignment), backbones))
    
    if mode == 'first':
        selected = list(markers)[:n]
    elif mode == 'last': 
        selected = list(markers)[-n:]
    elif mode in ['random', 'sample']:
        selected = random.sample(markers, k=n)
    else:
        raise ValueError
        
    
    circuit_permutations = []
    for assignments in selected:
        circuit_permutation = deepcopy(example)
        circuit_permutations.append(circuit_permutation)
        needs_assignment = get_needs_assigment(circuit_permutation)
        
        for part, (haploid, marker) in zip(needs_assignment, iter_key_items(assignments['selection'])):
            part['haploid'] = haploid
            part['backbone'] = find_backbone(marker, backbones)[1]
    return circuit_permutations
            
example = circuits[list(circuits)[1]]

now = arrow.now()
build_permutations = []
for circuit_name, parts in circuits.items():

    for i, perm_parts in enumerate(assign_markers(parts, n=3)):
        build_permutations.append({
            'name': circuit_name,
            'permutation': i,
            'created_at': str(now),
            'parts': perm_parts
        })

In [360]:
now = arrow.now()


filename = 'build_permutations_{}.json'.format(now)
with open(filename, 'w') as f:
    json.dump(build_permutations, f)

In [331]:
PROJECT = 'SD2 2021 Designs'

def create_oe_frag(session, name, frags, bb_frags):
    s1 = session.Sample.find_by_name(bb_frags[0])
    s2 = session.Sample.find_by_name(bb_frags[1])
    fragments = []
    missing = []
    for frag_name in frags:
        sample = session.Sample.find_by_name(frag_name)
        if not sample:
            missing.append(frag_name)
        else:
            fragments.append(sample)
    if missing:
        raise RuntimeError("missing {}".format(missing))
    if None in fragments:
        print(frags)
    length = 0
    for f in fragments:
        l = f.properties['Length']
        assert l
        length += l

    return {
        'id': None,
        'sample_type_id': session.SampleType.find_by_name('Fragment').id,
        'name': name,
        'Length': int(length),
        'Template': name,
        'project': PROJECT,
        'Yeast Marker': part['backbone']['marker'],
        'Forward Primer': s1.properties['Forward Primer'].name,
        'Reverse Primer': s2.properties['Reverse Primer'].name,
        'Fragment Mix Array': frags,
    }

def parse_to_fragment_build(session, part):
    if part['gate'] == 'Gate':
        i1 = part['inputs'][0]['name']
        i2 = part['inputs'][1]['name']
        o1 = part['outputs'][0]['name']
        f1 = '(SD2) pp2-pGRR-{}'.format(i1)
        f2 = '(SD2) {}{}-pGRR-ps'.format(i1, i2)
        f3 = 'PS-URGR-{}-tCyc1-TS'.format(o1)
        bb_frags = part['backbone']['fragments']
        
        frags = [f1, f2, f3] + list(bb_frags)
        
        name = "{locus}-{marker}<pGRR-{i1}{i2}-URGR-{o1}>".format(
            locus=part['backbone']['locus'],
            marker=part['backbone']['marker'],
            i1=i1,
            i2=i2,
            o1=o1
        )
        
        part['sample'] = create_oe_frag(session, name, frags, bb_frags)
    elif part['gate'] == 'Reporter':
        i1 = part['inputs'][0]['name']
        i2 = part['inputs'][1]['name']
        f1 = '(SD2) pp2-pGRR-{}'.format(i1)
        f2 = '(SD2) {}{}-pGRR-ps'.format(i1, i2)
        f3 = "PS-yeGFP-tCyc1-TS"
        bb_frags = part['backbone']['fragments']
        frags = [f1, f2, f3] + list(bb_frags)
        
        name = "{locus}-{marker}<pGRR-{i1}{i2}-yeGFP>".format(
            locus=part['backbone']['locus'],
            marker=part['backbone']['marker'],
            i1=i1,
            i2=i2,
        )
        part['sample'] = create_oe_frag(session, name, frags, bb_frags)
    elif part['gate'] == 'BE':
        o1 = part['outputs'][0]['name']
        name = 'pMOD8-pGALZ4-URGR-{}'.format(o1)
        part['sample'] = session.Sample.find_by_name(name).dump()
    elif part['gate'] == 'dCas9-Mxi1':
        name = 'pMOD4G-dcas9-mxi1'
        part['sample'] = session.Sample.find_by_name(name).dump()
    elif part['gate'] == 'dox_tf':
        name = 'pMOD-HO-KanMX-pREV1-rtTA-VPR'
        part['sample'] = session.Sample.find_by_name(name).dump()
    elif part['gate'] == 'be_tf':
        name = 'pMODKan-HO-pACT1-ZEV4'
        part['sample'] = session.Sample.find_by_name(name).dump()
    elif part['gate'] == 'DOX':
        f1 = 'pp2-7XTetO-pMinCyc1-PS'
        o1 = part['outputs'][0]['name']
        f2 = 'PS-URGR-{}-tCyc1-TS'.format(o1)
        bb_frags = part['backbone']['fragments']
        
        frags = [f1, f2] + list(bb_frags)
        
        name = "{locus}-{marker}<7XTetO-pMinCyc1-URGR-{o1}>".format(
            locus=part['backbone']['locus'],
            marker=part['backbone']['marker'],
            o1=o1
        )
        part['sample'] = create_oe_frag(session, name, frags, bb_frags)
    else:
        print('no parsing found for {}'.format(part['gate']))

with aq.with_cache() as sess:
    for circuit in build_permutations:
        for part in circuit['parts']:
            try:
                parse_to_fragment_build(sess, part)
            except Exception as e:
                print(e)

In [358]:
import json
from tqdm.auto import tqdm

def create_oe_fragments_from_build_perms(aq):
    
    new_fragments = {}
    perm_to_frag = {}
    frag_to_desc = {}
    for circuit in build_permutations:
        perm = circuit['permutation']
        for part in circuit['parts']:
            if part['sample']['id'] is None:
                sample = part['sample']
                new_fragments[sample['name']] = sample
                perm_to_frag.setdefault(perm, set())
                perm_to_frag[perm].add(sample['name'])

                desc = frag_to_desc.get(sample['name'], dict())
                desc.setdefault('circuits', list())
                desc['circuits'].append(circuit['name'] + "_" + str(perm))
                frag_to_desc[sample['name']] = desc
    frag_to_desc
    for frag in tqdm(list(new_fragments.values())):
        properties = {
                'Length': int(frag['Length']),
                'Fragment Mix Array': [aq.Sample.find_by_name(n) for n in frag['Fragment Mix Array']],
                'Forward Primer': aq.Sample.find_by_name(frag['Forward Primer']),
                'Reverse Primer': aq.Sample.find_by_name(frag['Reverse Primer']),
                'Yeast Marker': frag['Yeast Marker'],
                'Template': None,
                'Sequence': '',
            }
        new_sample = aq.Sample.new(
            name=frag['name'],
            sample_type_id=aq.SampleType.find_by_name('Fragment').id,
            description=filename,
            project=frag['project'],
            properties = properties
        )
        new_sample.raw_props = frag
        
        from_server = aq.Sample.find_by_name(new_sample.name)
        if from_server and from_server.id is not None:
            print('{} exists'.format(from_server.name))
            continue
            
        new_sample.save()
        print('saving "{}"'.format(new_sample.name))
        
        from_server = aq.Sample.find_by_name(new_sample.name)
        
        from_server.properties['Template'] = from_server
        from_server.save()
        
create_oe_fragments_from_build_perms(aq)



  0%|          | 0/28 [00:00<?, ?it/s]

HIS-HIS<7XTetO-pMinCyc1-URGR-W8> exists
URA-URA<pGRR-W10W8-URGR-W5> exists
ARS314-bleo<pGRR-W5W5-URGR-W20> exists
LEU-LEU<pGRR-W5W5-URGR-W36> exists
URA-URA<pGRR-W20W36-yeGFP> exists
ARS314-bleo<pGRR-W10W8-URGR-W5> exists
TRP-TRP<pGRR-W5W5-URGR-W20> exists
saving "HIS-HIS<7XTetO-pMinCyc1-URGR-W34>"
saving "URA-URA<pGRR-W8W34-URGR-W17>"
saving "ARS314-bleo<pGRR-W17W17-URGR-W10>"
saving "LEU-LEU<pGRR-W17W17-URGR-W20>"
saving "URA-URA<pGRR-W10W20-yeGFP>"
saving "ARS314-bleo<pGRR-W8W34-URGR-W17>"
saving "TRP-TRP<pGRR-W17W17-URGR-W10>"
saving "ARS314-bleo<pGRR-W10W8-URGR-W36>"
saving "URA-URA<pGRR-W5W36-yeGFP>"
saving "TRP-TRP<pGRR-W10W8-URGR-W36>"
saving "URA-URA<pGRR-W8W34-URGR-W10>"
saving "URA-URA<pGRR-W10W17-yeGFP>"
saving "ARS314-bleo<pGRR-W8W34-URGR-W10>"
saving "TRP-TRP<pGRR-W8W34-URGR-W17>"
saving "HIS-HIS<7XTetO-pMinCyc1-URGR-W5>"
saving "URA-URA<pGRR-W36W5-yeGFP>"
saving "URA-URA<pGRR-W17W34-yeGFP>"
saving "URA-URA<pGRR-W5W8-URGR-W36>"
saving "URA-URA<pGRR-W36W36-yeGFP>"
saving "